# TensorFlow 패키지 소개

TensorFlow 패키지는 Theano 패키지와 마찬가지로 선형 대수 심볼 컴파일러(Symbolic Linear Algebra Compiler)이다. Theano에 비해 분산처리 기능이 강화되어 있다.

## TensorFlow 기본 사용법

Theano와 유사하게 TensorFlow도 다음과 같은 과정을 거쳐 사용한다.

1. 심볼 변수 정의
2. 심볼 관계 정의
3. 세션 정의
4. 세션 사용

세션(Session)은 Theano의 함수와 유사한 역할을 하며 실제 심볼 변수의 관계를 분석하고 값을 계산해준다.

### 심볼 변수 정의

TensorFlow는 `Variable` 명령으로 정의하며 Theano와 달리 각 심볼 변수의 형태를 스칼라, 벡터, 행렬 등으로 명시적으로 정의하지 않는 대신 초기값을 이용해서 형태를 지정한다.

In [3]:
import tensorflow as tf

In [4]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

In [3]:
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

### 심볼 관계 정의

이미 만들어진 심볼 변수에 일반 사칙연산이나 TensorFlow 수학 함수를 사용하여 종속 변수 역할을 하는 심볼 변수를 만들 수 있다.

In [4]:
z = x + y

In [5]:
type(z)

tensorflow.python.framework.ops.Tensor

In [6]:
u = tf.log(z)

In [7]:
type(u)

tensorflow.python.framework.ops.Tensor

### 세션

TensorFlow의 세션은 Theano의 함수와 유사한 역할을 하며 실제로 계산 그래프를 생성하고 값을 계산하기위한 환경을 제공한다. Theano의 함수와 달리 세션 생성과 실행 시작, 종료를 명시해야 한다. 
* tensorflow는 다른 컴퓨터와 통신을 하여 여러개의 컴퓨터를 동시에 사용하도록 만든 프로그램이다. 그래서 세션을 시작하면 나중에 꼭 꺼줘야 한다

* 세션 생성: `Session` 객체 생성
* 세션 사용: `run` 메서드. 실제로 값이나 expression을 계산한다.(evaluation)
* 세션 종료: `close` 메서드. with 문으로 대체하기도 한다.

In [8]:
sess = tf.Session()
init = tf.global_variables_initializer() ## initializing 하는 그래프 
sess.run(init) ## 스페셜 그래프 실행, 각변수들의 초기값이 들어감
print(sess.run(z)) ## 1+2 
print(sess.run(u)) ## log(1+2) 

3.0
1.09861


### 미분

TensorFlow 도 심볼릭 연산에 의한 미분 계산이 가능하다.

In [9]:
f = x ** 2
fx = tf.gradients(f, [x]) ## []을 쓰면 나중에 자동으로 세션을 꺼줌 

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(f))    
    print(sess.run(fx))

1.0
[2.0]


### 최적화

TensorFlow에는 Theano와 달리 최적화를 위한 `GradientDescentOptimizer` 등의 클래스가 미리 구현되어 있으므로 사용자가 구현할 필요가 없다.
자동으로 optimization 하는 그래프를 자동으로 만들수 있다 

## 예제: 선형 회귀

* https://www.tensorflow.org/versions/master/get_started/index.html 을 기반으로 tensorboard 용 로그 코드 추가

In [5]:
##스타트업 파일

from scipy.stats import norm
import numpy as np
import scipy as sp
from pandas.core import datetools 
import pandas as pd
import statsmodels.api as sm ## R 패키지에서 불러온것, 샘플데이터 옮겨쓸수 있는 프로젝트 있음. 회귀분석에 유용 
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk 
## 전처리 

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pylab 
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  


In [6]:
# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))#1로 정해줌 , 유니폼하게 -1 에서 1사이 사이즈를 정해줌
b = tf.Variable(tf.zeros([1])) ## b는 0으로 정해줌
y = W * x_data + b ## 정확하게는 y-hat임 

weights_summary = tf.summary.histogram("weights", W)
biases_summary = tf.summary.histogram("biases", b)
y_summary = tf.summary.histogram("y", y)
merged = tf.summary.merge([weights_summary, biases_summary, y_summary])

loss = tf.reduce_mean(tf.square(y - y_data)) ## loss라는 그래프만듬 = y-y hat 의 제곱 
optimizer = tf.train.GradientDescentOptimizer(0.025) ## learning rate = 0.025
train = optimizer.minimize(loss) ## minimize method 실행 
## 여기서 train만 실행시키면 loss, optimizer 모두 자동으로 만들어짐 (x,y도 모두 실행됨 )

sess = tf.Session() ## 세션

init = tf.global_variables_initializer() ## 초기화함 
sess.run(init)

writer = tf.summary.FileWriter("/home/dockeruser/logs", sess.graph)

for step in range(1001): ## 1000번 돌림 
    sess.run(train)
    if step % 20 == 0:
        summary = sess.run(merged)
        writer.add_summary(summary, step)
        print(step, sess.run(loss), sess.run(W), sess.run(b))
        
sess.close()## 세션 클로징 


0 0.0857465 [ 0.08173294] [ 0.0155838]
20 0.00747341 [ 0.15865117] [ 0.1883246]
40 0.00114221 [ 0.17630877] [ 0.23915505]
60 0.000568797 [ 0.17747679] [ 0.25539723]
80 0.000463805 [ 0.17422844] [ 0.26174241]
100 0.000403573 [ 0.16996078] [ 0.26517785]
120 0.000353342 [ 0.16561939] [ 0.26768565]
140 0.000309538 [ 0.16145781] [ 0.26983288]
160 0.000271179 [ 0.15753488] [ 0.27178609]
180 0.000237574 [ 0.15385526] [ 0.27359834]
200 0.000208134 [ 0.15040889] [ 0.27529007]
220 0.000182342 [ 0.14718252] [ 0.27687222]
240 0.000159746 [ 0.14416249] [ 0.27835277]
260 0.00013995 [ 0.14133573] [ 0.27973843]
280 0.000122607 [ 0.13868991] [ 0.28103533]
300 0.000107414 [ 0.13621341] [ 0.28224924]
320 9.41031e-05 [ 0.13389544] [ 0.28338543]
340 8.24417e-05 [ 0.13172583] [ 0.28444895]
360 7.22255e-05 [ 0.12969509] [ 0.28544432]
380 6.32755e-05 [ 0.12779437] [ 0.286376]
400 5.54344e-05 [ 0.12601531] [ 0.28724808]
420 4.85649e-05 [ 0.1243501] [ 0.2880643]
440 4.25468e-05 [ 0.12279148] [ 0.28882825]
460 3

## TensorBoard

TensorFlow는 theano와 달리 모형 내부와 결과를 모니터링 할 수 있는 TensorBoard 라는 웹서버를 제공한다.

TensorBoard 웹서버는 포트 6006을 사용하므로 만약 도커를 사용하는 경우에는 다음과 같이 포트를 열고 실행해야 한다.

내꺼 Docker Quickstarter에서 하기의 명령어 실행 (만약 미리 rpython이 실행이 되어있다면 stop 시켜주고 rpython 디렉토리 삭제 후 다시하기의 명령실행) 

```
docker run --name rpython -it -p 8888:8888 -p 6006:6006 datascienceschool/rpython
```

### tensorboard 용 로그 생성

tensorboard는 tensorflow 처리의 결과로 생성된 로그 파일을 읽어서 웹 화면에 나타낼 뿐이다. 따라서 계산 코드에서 로그 생성을 위한 코드를 추가해 주어야 한다. 

로그 코드는 다음과 같은 순서로 추가한다.

1. `summary.histogram` 명령으로 개별 변수 기록용 summary 생성
2. `summary.merge` 명령으로 개별 변수 기록용 summary를 하나의 summary로 합침
3. 세션 생성후 `tf.summary.FileWriter` 명령으로 기록용 writer 객체 생성(위 예제에서는 `writer`). 이 때 기록할 디렉토리 설정
4. `sess.run` 명령으로 summary 객체(위 예제에서는 `merged`)를 보내면 기록 실시
5. 기록용 write 객체의 `add_summary` 명령으로 summary를 실제 파일에 쓰기.

### tensorboard 가동

TensorBoard를 가동하기 위해서는 콘솔에서 다음과 같이 로그 디렉토리를 설정하여 실행한다.

```
$ tensorboard --logdir=/home/dockeruser/logs
```

위와 같이 실행한 다음에는 다음 주소로 연결하면 TensorBoard 화면을 볼 수 있다.

http://192.168.99.100:6006


<img src="https://datascienceschool.net/upfiles/3fd708a2121d4bb5887b29a7b4d320a6.png" style="width:100%;">